<center><p style="font-size:30px;font-family:consolas">SOUNDCLOUD - Exploratory Data Analysis</p></center>

In [ ]:
!pip install selenium
!pip install webdriver-manager

We have to install:
- **Selenium**: use for working with browse automatically
- **Webdriver-manager**: use for install Chronium without installing Chronium before

In [1]:
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import time
import datetime as dt

import csv
import random

from concurrent.futures import ThreadPoolExecutor

# 1. Data collection: 

## 1.1. Initialize: 

Function `initialize_driver` use for initialize some Chrome options (ex: not load image, headless mode,...) and open specified url.

In [2]:
# Initialize webdriver
def initialize_driver(url):
    chrome_options = webdriver.ChromeOptions()
    prefs={"profile.managed_default_content_settings.images": 2, 'disk-cache-size': 4096 }
    chrome_options.add_experimental_option("prefs", prefs) # Manage image loading and run on disk cache
    chrome_options.add_argument("--headless") # Runs Chrome in headless mode
    chrome_options.add_argument('--no-sandbox') # Bypass OS security model
    chrome_options.add_argument('--disable-dev-shm-usage') # overcome limited resource problems
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)
    driver.get(url)
    return driver

`scrollWebpage` use for scrolling webside

In [3]:
def scrollWebpage(n_times, driver):
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")
    for _ in range(n_times):
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        # Wait to load page
        time.sleep(2)
        
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        # compare 2 height (to check whether reach the end of website)
        if new_height == last_height:
            # webpage may be loading
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
        
        # update height
        last_height = new_height

below function use for accept cookies if it exists

In [4]:
def accept_cookies(driver):
    try:
      WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH,
                                                        '//button[@id="onetrust-accept-btn-handler"]')))
    except Exception as err:
      print("there's no cookies")
    else:
      ck = driver.find_element_by_xpath('//button[@id="onetrust-accept-btn-handler"]') # accept cookies
      driver.execute_script("arguments[0].click();", ck)

convert string to number (because almost crawled data from web are string type)

In [5]:
def string2number(numstr):
    if numstr == "Repost" or numstr == "Like":
        return 0

    numstr = numstr.replace(',', '')
    mul = 1
    if numstr[-1] == 'K':
        numstr = numstr.replace('K', '')
    elif numstr[-1] == 'M':
        numstr = numstr.replace('M', '')
        mul = 1000000
    elif numstr[-1] == 'B':
        numstr = numstr.replace('B', '')
        mul = 1000000000
    return int(float(numstr) * mul)

Function `write_csv_file` use for writing a list data to csv file

In [6]:
def write_csv_file(filename, header, content):
    with open(filename, 'w', encoding='utf-8-sig', newline='') as file:
        writter = csv.writer(file, delimiter=',')
        writter.writerow(header)
        for l in content:
            writter.writerow(l)

Initialize some common variable for some task (assign, storage)

In [7]:
# Initialize some ultilities
sleep = 3

searchStr = 'abcdefghijklmnoprstwuyxyz123456789'
base_link = 'https://www.soundcloud.com'

In [8]:
# Some variable for assigning and storing
users_id = dict() # url : id
users_list = [] # all data
users_id_assign = 0 # use for assigning id
unknow_users = dict() # url : [id, username] --> use temporary store un-stored users

playlists_id = dict()
playlists_list = []
playlists_id_assign = 0

tracks_id = dict() # url : [id, trackname]
tracks_list = []
tracks_id_assign = 0

## 1.2. Crawling users (initialize some functions to crawl user data): 

In [9]:
def CrawlUsersData_from_search(page_source):
    soup = BeautifulSoup(page_source, 'html.parser')
    users = soup.find_all('div', class_="userItem sc-media g-flex-row-centered sc-px-2x sc-py-1x m-horizontal m-verified")
    
    global users_id_assign
    for user in users:
        user_url = base_link + user.find('a', class_="userItem__coverArt sc-media-image sc-mr-2x")['href']
        if user_url in users_id.keys():
            continue
        
        username = user.find('a', class_="sc-link-dark sc-link-primary").text.strip().encode('utf-8').decode('utf-8')
        
        confirm_owner = user.find('span', class_="sc-status-icon sc-status-icon-verified sc-status-icon-small sc-ir")
        if confirm_owner:
            confirm_owner = 'Yes'
        else:
            confirm_owner = 'Not yet'
        
        user_followers = user.find('li', class_="sc-ministats-item")
        if user_followers:
            user_followers = int(user.find('li', class_="sc-ministats-item")['title'].split(' ')[0].replace(',', ''))
        else:
            user_followers = 0
        
        user_detail = user.find('h3', \
                                class_='userItem__details sc-type-light sc-text-secondary sc-text-h4 sc-mt-0.5x')\
                                .find_all('div')
        detail_name, country = tuple(map(lambda x : x.text.strip().encode('utf-8').decode('utf-8'), user_detail))
        
        user_id = users_id_assign
        users_id_assign += 1
        users_id[user_url] = user_id
        
        users_list.append([user_id, username, user_url, confirm_owner, detail_name, country, user_followers])

In [10]:
def CrawlUsersData_from_page(url):
    driver = initialize_driver(url)
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH,
                                                            '//button[@id="onetrust-accept-btn-handler"]')))
    ck = driver.find_element_by_xpath('//button[@id="onetrust-accept-btn-handler"]')
    driver.execute_script("arguments[0].click();", ck)
    page_source = driver.page_source
    driver.quit()
    soup = BeautifulSoup(page_source, 'html.parser')
    
    username = unknow_users[url][0]
    
    if soup.find('span', class_="verifiedBadge userDropbar__verifiedBadge"):
        confirm_owner = 'Yes'
    else:
        confirm_owner = 'Not yet'
        
    user_followers = int(soup.find('a', class_="infoStats__statLink sc-link-light sc-link-primary")['title']\
                         .split(' ')[0].replace(',', ''))

    detail_name = soup.find('h3', 
                            class_="profileHeaderInfo__additional g-type-shrinkwrap-block theme-dark g-type-shrinkwrap-large-secondary")
    if detail_name:
        detail_name = detail_name.text.strip().encode('utf-8').decode('utf-8')
    else:
        detail_name = ''
        
    country = soup.find('h3', 
                        class_='profileHeaderInfo__additional g-type-shrinkwrap-block theme-dark g-type-shrinkwrap-large-secondary sc-mt-1x')
    if country:
        country = country.text.strip().encode('utf-8').decode('utf-8')
    else:
        country = ''

    user_id = unknow_users[url][1]

    users_list.append([user_id, username, url, confirm_owner, detail_name, country, user_followers])

## 1.3. Crawling playlists (initialize some functions to crawl playlist data) : 

In [11]:
# This funciton is temporarily unavailable
def CrawlPlaylistData_from_page(url):
    global playlists_id_assign
    global users_id_assign
    global tracks_id_assign
    
    # open driver
    driver = initialize_driver(url)
    accept_cookies(driver)
    scrollWebpage(1000, driver)
    page_source = driver.page_source
    driver.close()
    driver.quit()
    soup = BeautifulSoup(page_source, 'html.parser')
    
    # crawl
    pll_name = soup.find('h1', 
                         class_='soundTitle__title sc-font g-type-shrinkwrap-inline g-type-shrinkwrap-large-primary theme-dark')\
                        .text.strip().encode('utf-8').decode('utf-8')
    #crawl user information
    pll_user = soup.find('a', 
                         class_='userBadge__usernameLink sc-link-dark sc-link-primary sc-truncate sc-mr-0.5x')
    pll_username = pll_user.text.strip().encode('utf-8').decode('utf-8')
    pll_user_url = pll_user['href']
    if pll_user_url not in users_id.keys() and pll_user_url not in unknow_users.keys():
        unknow_users['pll_url'] = [users_id_assign, pll_username]
        users_id_assign += 1
        
    pll_time = soup.find('time', class_='relativeTime')['datetime']
    
    pll_tags = soup.find_all('span', class_='sc-truncate sc-tagContent')
    if pll_tags:
        pll_mainTag = pll_tags[0].text.strip().encode('utf-8').decode('utf-8')
        pll_detailTags = list(map(lambda x : x.text.strip().encode('utf-8').decode('utf-8'), pll_tags[1:]))
    else:
        pll_mainTag = ''
        pll_detailTags = []
    
    pll_stat = soup.find_all('li', class_='sc-ministats-item')
    pll_like = int(pll_stat[0]['title'].strip().split(' ')[0].replace(',', ''))
    pll_repost = int(pll_stat[1]['title'].strip().split(' ')[0].replace(',', ''))
    
    # crawl tracks
    tracks = soup.find_all('div', class_='trackItem__content sc-truncate')
    pll_tracks_id = []
    for track in tracks:
        track_url = base_link + track.find('a',
                                          class_='trackItem__trackTitle sc-link-dark sc-link-primary sc-font-light')['href']\
                                            .split('?')[0]
        if track_url not in tracks_id.keys():
            track_id = tracks_id_assign
            tracks_id_assign += 1
            tracks_id[track_url] = track_id
        else:
            track_id = tracks_id[track_url]
        
        pll_tracks_id.append(track_id)
        
        pll_url = str(url)
    playlists_list.append([pll_name, pll_url, playlists_id[url], pll_username, pll_time, pll_mainTag, 
                     pll_detailTags, pll_like, pll_repost, len(pll_tracks_id), pll_tracks_id])

In [12]:
def CrawlPlaylistData_from_search(page_source):
    soup = BeautifulSoup(page_source, 'html.parser')

    global playlists_id_assign
    global tracks_id_assign
    global users_id_assign
    # print(len(playlists))

    playlists = soup.find_all('div', class_='sound__body')
    for playlist in playlists:
        playlist_base = playlist.find('a', class_='sc-link-primary soundTitle__title sc-link-dark sc-text-h4')
        playlist_url = base_link + playlist_base['href']
        if playlist_url in playlists_id.keys():
            continue
        
        playlist_available = playlist.find('span', class_='g-geoblocked-icon')
        if playlist_available:
            continue
        
        playlist_id = playlists_id_assign
        playlists_id[playlist_url] = playlist_id
        playlists_id_assign += 1
        
        playlist_name = playlist_base.text.strip().encode('utf-8').decode('utf-8')

        playlist_time = playlist.find('time', class_="relativeTime sc-text-secondary sc-text-captions")['datetime']

        playlist_tag = playlist.find('span', class_='sc-truncate sc-tagContent')
        if playlist_tag:
            playlist_tag = playlist_tag.text.strip().encode('utf-8').decode('utf-8')
        else:
            playlist_tag = ''

        playlist_like = string2number(playlist.find('button', class_="sc-button-like sc-button-secondary sc-button sc-button-small sc-button-responsive").text)

        playlist_repost = string2number(playlist.find('button', class_="sc-button-repost sc-button-secondary sc-button sc-button-small sc-button-responsive").text)

        # crawl user
        playlist_username = playlist.find('span', class_='soundTitle__usernameText').text.strip().encode('utf-8').decode('utf-8')
        playlist_user_url = base_link + playlist.find('span', class_='soundTitle__usernameText').parent['href']

        if playlist_user_url not in users_id:
            user_id = users_id_assign
            users_id[playlist_user_url] = user_id
            users_id_assign += 1

            users_list.append([user_id, playlist_username, playlist_user_url, '', '', '', ''])
        else:
            user_id = users_id[playlist_user_url]

        # crawl tracks
        playlist_tracks = playlist.find_all('div', class_='compactTrackListItem sc-media sc-border-light-bottom clickToPlay m-interactive m-playable')
        playlist_tracks_id = []
        for track in playlist_tracks:
            # this is new version
            track_url = base_link + track.find('span', class_='compactTrackListItem__trackTitle sc-text-primary sc-text-h4')['data-permalink-path'].split('?')[0]
            if track_url not in tracks_id:
                track_id = tracks_id_assign
                tracks_id[track_url] = track_id
                tracks_id_assign += 1
            else:
                track_id = tracks_id[track_url]
                
            playlist_tracks_id.append(track_id)

        playlists_list.append([playlist_id, playlist_name, playlist_url, user_id, playlist_username, playlist_time, playlist_tag, 
                                playlist_like, playlist_repost, len(playlist_tracks_id), playlist_tracks_id])

## 1.4. Crawling tracks (initialize function to crawl tracks data):

In [13]:
def CrawlingTracksData_from_page(url):
    driver = initialize_driver(url)
    accept_cookies(driver)
    scrollWebpage(10, driver)
    page_source = driver.page_source
    driver.close()
    driver.quit()
    soup = BeautifulSoup(page_source, 'html.parser')

    track_name = soup.find('h1', class_="soundTitle__title sc-font g-type-shrinkwrap-inline g-type-shrinkwrap-large-primary theme-dark")\
                    .text.strip().encode('utf-8').decode('utf-8')
    track_username = soup.find('a', class_="sc-link-secondary").text.strip().encode('utf-8').decode('utf-8')
    
    track_time = soup.find('time', class_='relativeTime')['datetime']

    track_tag = soup.find('a', class_="sc-tag sc-tag-large")
    if track_tag:
        track_tag = track_tag.text.strip().encode('utf-8').decode('utf-8')
    else:
        track_tag = ''
    
    track_stat = soup.find('ul', class_="soundStats sc-ministats-group listenEngagement__stats sc-ministats-group-right")
    track_play, track_like, track_repost = tuple(map(lambda x : int(x['title'].strip().split(' ')[0].replace(',', '')),
                                                    track_stat.find_all('li', class_="sc-ministats-item")))

    tracks_list.append([tracks_id[url], track_name, url, track_time, track_tag, track_play, track_like, track_repost])    

## 1.5. Crawling (start crawling):

In [14]:
people_search_str = [base_link + '/search/people?q=' + c for c in searchStr]
# page_sources = []
def get_users(url):
    driver = initialize_driver(url)
    accept_cookies(driver)
    scrollWebpage(5, driver)
    page_source = driver.page_source
    driver.close()
    driver.quit()
    
    CrawlUsersData_from_search(page_source)

In [15]:
with ThreadPoolExecutor(max_workers=15) as executor:
    executor.map(get_users, people_search_str)

C:\Users\DELL\AppData\Local\Temp\ipykernel_4892\2889666167.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)
C:\Users\DELL\AppData\Local\Temp\ipykernel_4892\3544239256.py:8: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  ck = driver.find_element_by_xpath('//button[@id="onetrust-accept-btn-handler"]') # accept cookies


In [16]:
playlists_search_str = [base_link + '/search/sets?q=' + c for c in searchStr]

def get_playlists(url):
    driver = initialize_driver(url)
    accept_cookies(driver)
    scrollWebpage(7, driver)

    try:
        more_tracks_button = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, '//a[@class="compactTrackList__moreLink sc-link-light sc-link-primary sc-border-light sc-text-h4"]')))
    except Exception as err:
        print('Something went wrong!')
    else:
        for bt in more_tracks_button:
            driver.execute_script("arguments[0].click();", bt)
    
    page_source = driver.page_source
    driver.close()
    driver.quit()

    CrawlPlaylistData_from_search(page_source)

In [17]:
with ThreadPoolExecutor(max_workers=15) as executor:
    executor.map(get_playlists, playlists_search_str)

C:\Users\DELL\AppData\Local\Temp\ipykernel_4892\2889666167.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)
C:\Users\DELL\AppData\Local\Temp\ipykernel_4892\3544239256.py:8: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  ck = driver.find_element_by_xpath('//button[@id="onetrust-accept-btn-handler"]') # accept cookies


there's no cookies
there's no cookies
there's no cookies
there's no cookies
there's no cookies
there's no cookies
there's no cookies
there's no cookies
there's no cookies
there's no cookies


In [18]:
crawl_tracks_url = random.choices(list(tracks_id.keys()), k=1100)

In [19]:
with ThreadPoolExecutor(max_workers=15) as executor:
    executor.map(CrawlingTracksData_from_page, crawl_tracks_url)

C:\Users\DELL\AppData\Local\Temp\ipykernel_4892\2889666167.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)


there's no cookies
there's no cookiesthere's no cookies

there's no cookies
there's no cookies
there's no cookies
there's no cookies
there's no cookies
there's no cookies
there's no cookies


C:\Users\DELL\AppData\Local\Temp\ipykernel_4892\3544239256.py:8: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  ck = driver.find_element_by_xpath('//button[@id="onetrust-accept-btn-handler"]') # accept cookies


there's no cookies
there's no cookies
there's no cookies
there's no cookies
there's no cookies
there's no cookies
there's no cookies
there's no cookies
there's no cookies
there's no cookies
there's no cookies
there's no cookies
there's no cookies
there's no cookies


In [20]:
user_header = ['user_id', 'username', 'user_url', 'confirm_owner', 'detail_name', 'country', 'user_followers']
write_csv_file('./user.csv', user_header, users_list)

In [21]:
playlist_header = ['playlist_id', 'playlist_name', 'playlist_url', 'owner_id', 'username', 'playlist_time', 'playlist_tag', 
                                'like', 'repost', 'size', 'tracks']
write_csv_file('./playlist.csv', playlist_header, playlists_list)

In [22]:
track_header=['track_id', 'track_name', 'url', 'datetime', 'track_tag', 'track_play', 'track_like', 'track_repost']
write_csv_file('./track.csv', track_header, tracks_list)